# Análise explorátoria dos dados de treino/teste

https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets

In [ ]:
import os
import shutil
import duckdb
import tabulate
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt

from warnings import filterwarnings
filterwarnings('ignore')

Carregar os dados pré-processados obtidos do repositorio do kaggle

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive

  os.makedirs('./datasets', exist_ok=True)
  drive.mount('/content/drive')

  shutil.copy('/content/drive/MyDrive/DataScience/Analytics/Estudo Fraude/df_notarget.parquet'  ,'/content/datasets/')
  shutil.copy('/content/drive/MyDrive/DataScience/Analytics/Estudo Fraude/df_train_test.parquet','/content/datasets/')
  shutil.copy('/content/drive/MyDrive/DataScience/Analytics/Estudo Fraude/df_validation.parquet','/content/datasets/')

Carregar os dados de treino e teste

In [ ]:
df = pd.read_parquet('./datasets/df_train_test.parquet', engine='pyarrow')
df.head()

In [ ]:
df['id'] = df['id'].astype(str)
df['client_id'] = df['client_id'].astype(str)
df['card_id'] = df['card_id'].astype(str)
df['merchant_id'] = df['merchant_id'].astype(str)
df['zip'] = df['zip'].astype(str)
df['card_number'] = df['card_number'].astype(str)
df['cvv'] = df['cvv'].astype(str)
df['birth_year'] = df['birth_year'].astype(str)
df['birth_month'] = df['birth_month'].astype(str)


1. **id**: Identificador único da transação.
2. **date**: Data em que a transação foi realizada.
3. **client_id**: Identificador único do cliente.
4. **card_id**: Identificador único do cartão utilizado na transação.
5. **amount**: Valor da transação.
6. **use_chip**: Indicador se a transação foi efetuada com o uso de chip (sim/não).
7. **merchant_id**: Identificador do comerciante onde a transação foi realizada.
8. **merchant_city**: Cidade onde está localizado o comerciante.
9. **merchant_state**: Estado onde está localizado o comerciante.
10. **zip**: Código postal do local do comerciante.
11. **mcc**: Código de Categoria do Comerciante, que indica o tipo de negócio ou setor do comerciante.
12. **errors**: Indicador de erros ocorridos durante a transação.
13. **description**: Descrição ou comentário associado à transação.
14. **target**: Indicador se a transação foi fraudulenta (Yes) ou não (No).
15. **current_age**: Idade atual do cliente.
16. **retirement_age**: Idade de aposentadoria do cliente.
17. **birth_year**: Ano de nascimento do cliente.
18. **birth_month**: Mês de nascimento do cliente.
19. **gender**: Gênero do cliente.
20. **address**: Endereço residencial do cliente.
21. **latitude**: Latitude do endereço do cliente.
22. **longitude**: Longitude do endereço do cliente.
23. **per_capita_income**: Renda per capita do cliente ou região.
24. **yearly_income**: Renda anual do cliente.
25. **total_debt**: Dívida total do cliente.
26. **credit_score**: Pontuação de crédito do cliente.
27. **num_credit_cards**: Número de cartões de crédito que o cliente possui.
28. **card_brand**: Bandeira do cartão utilizado na transação (Visa, Mastercard, etc.).
29. **card_type**: Tipo do cartão (crédito, débito, etc.).
30. **card_number**: Número do cartão utilizado na transação.
31. **expires**: Data de expiração do cartão.
32. **cvv**: Código de segurança do cartão.
33. **has_chip**: Indicador se o cartão possui chip (sim/não).
34. **num_cards_issued**: Número de cartões emitidos para o cliente.
35. **credit_limit**: Limite de crédito do cartão.
36. **acct_open_date**: Data de abertura da conta do cliente.
37. **year_pin_last_cha**: Ano da última alteração do PIN do cartão.
38. **card_on_dark_web**: Indicador se o cartão foi encontrado em listas de venda na dark web.

In [ ]:
def get_outliers(df, column, min_zero = True):
  Q1 = df[column].quantile(0.25)
  Q3 = df[column].quantile(0.75)
  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  if min_zero:
    lower_bound = lower_bound if lower_bound >= 0 else 0
  upper_bound = Q3 + 1.5 * IQR
  if min_zero:
    upper_bound = upper_bound if upper_bound >= 0 else 0

  return lower_bound, upper_bound

In [ ]:
def plot_hist_box(df_, var):
  data = df[var]

  fig = plt.figure(figsize=(15,4))
  ax1 = fig.add_subplot(1, 3, 1)
  ax1.set_title('Histograma')
  sns.histplot(data, kde=True, ax = ax1)
  ax1.set_xlabel(var)
  ax1.tick_params(axis='x', rotation=90)
  ax1.set_ylabel('Frequência')

  ax2 = fig.add_subplot(1, 3, 2)
  ax2.set_title('Boxplot')
  sns.boxplot(data, ax=ax2)
  ax2.set_ylabel('Frequência')

  ax3 = fig.add_subplot(1, 3, 3)
  ax3.set_title('QQ-plot')
  sm.qqplot(data, line='s', ax=ax3)

  plt.suptitle(f'{var}')
  plt.tight_layout()
  plt.show()

In [ ]:
def descritiva(df_, var, vresp: str, max_classes=5, map_target: dict|None = None):
    df = df_[[var, vresp]].copy()

    if df[var].dtype in [np.float64, np.int64]:
      if df[var].nunique()>max_classes:
          df[var] = pd.qcut(df[var], max_classes, duplicates='drop')

    if map_target is not None:
      df[vresp] = df[vresp].map(map_target)

    fig, ax1 = plt.subplots(figsize=(10, 6))

    ax1 = sns.countplot(data=df, x=var, palette='viridis', alpha=0.5, ax=ax1)
    ax1.set_ylabel('Frequência', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    for p in ax1.patches:
        ax1.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    ax2 = ax1.twinx()

    sns.pointplot(data=df, y=vresp, x=var, ax=ax2)
    ax2.set_ylabel(vresp, color='red')

    ax1.set_zorder(2)
    ax1.patch.set_visible(False)  # Tornar o fundo do eixo 1 transparente

    # Exibir o gráfico
    plt.show()
    del df

## Análise Temporal

Vamos analisar os dados com base na caracteristica tempo a fim de entender as caracteristicas das transações

### Análise diaria

In [ ]:
df_time_day = pd.DataFrame()
df_time_day['day'] = df['date'].dt.to_period('D')
df_time_day['target'] = df['target'].apply(lambda x: 1 if x == 'Yes' else 0)
df_time_day['amount'] = df['amount']
df_time_day.head()

In [ ]:
df_transactions_day = df_time_day.groupby(by='day').agg({'amount': ['sum', 'count'], 'target': 'sum'}).reset_index()
df_transactions_day.columns = ['day', 'amount', 'transactions', 'total_frauds']
df_transactions_day['mean_amount'] = (df_transactions_day['amount'] / df_transactions_day['transactions']).round(2)
df_transactions_day['has_fraud'] = df_transactions_day['total_frauds'] > 0
df_transactions_day.head()

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(221)

plt.title('Distribuição das quantidades de transações diarias')
ax = sns.histplot(df_transactions_day['transactions'], kde=True)
ax.vlines(df_transactions_day['transactions'].mean(), 0, ax.get_ylim()[1], color='red', linestyles='dashed', label=f'Média {round(df_transactions_day["transactions"].mean(), 2)}')
ax.vlines(df_transactions_day['transactions'].median(), 0, ax.get_ylim()[1], color='green', linestyles='dashed', label=f'Mediana {round(df_transactions_day["transactions"].median(), 2)}')
ax.legend()

plt.subplot(222)
plt.title('Distribuição das médias de transações diarias')
ax = sns.histplot(df_transactions_day['mean_amount'], kde=True)
ax.vlines(df_transactions_day['mean_amount'].mean(), 0, ax.get_ylim()[1], color='red', linestyles='dashed', label=f'Média {round(df_transactions_day["mean_amount"].mean(), 2)}' )
ax.legend()

plt.subplot(223)
plt.title('Distribuição das quantidades de fraudes diarias')
ax = sns.histplot(df_transactions_day.query('total_frauds > 0')['total_frauds'], kde=True)
ax.vlines(df_transactions_day.query('total_frauds > 0')['total_frauds'].mean(), 0, ax.get_ylim()[1], color='red', linestyles='dashed', label=f'Média {round(df_transactions_day.query("total_frauds > 0")["total_frauds"].mean(), 2)}')
ax.vlines(df_transactions_day.query('total_frauds > 0')['total_frauds'].median(), 0, ax.get_ylim()[1], color='green', linestyles='dashed', label=f'Mediana {round(df_transactions_day.query("total_frauds > 0")["total_frauds"].median(), 2)}')
ax.legend()

plt.subplot(224)
plt.title('Proporção de dias com fraudes')
ax = sns.countplot(data=df_transactions_day, x='has_fraud')

total = len(df_transactions_day)
for path in ax.patches:
  ax.annotate(f'{path.get_height()/total:.2f}', (path.get_x() + path.get_width() / 2., path.get_height()), ha='center', va='center', xytext=(0, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(131)
plt.title('Outliers da quantidade de transações diarias')
ax = sns.boxplot(x=df_transactions_day['transactions'])

plt.subplot(132)
plt.title('Outliers da média de transações diarias')
ax = sns.boxplot(x=df_transactions_day['mean_amount'])
ax.set_xlim(0, 100)

plt.subplot(133)
plt.title('Outliers da quantidade de fraudes diarias')
ax = sns.boxplot(x=df_transactions_day['total_frauds'])

plt.tight_layout()
plt.show()

In [ ]:
stat, p = stats.shapiro(df_transactions_day['transactions'])

alpha = 0.05

if p > alpha:
    print('A amostra provavelmente segue uma distribuição normal (falha ao rejeitar H0)')
else:
    print('A amostra provavelmente não segue uma distribuição normal (rejeita H0)')

In [ ]:
min, max = get_outliers(df_transactions_day, 'transactions')
print('Outlier min:', min)
print('Outlier max:', max)

In [ ]:
min, max = get_outliers(df_transactions_day, 'mean_amount')
print('Outlier min:', round(min, 2))
print('Outlier max:', round(max, 2))

In [ ]:
min, max = get_outliers(df_transactions_day, 'total_frauds')
print('Outlier min:', round(min, 2))
print('Outlier max:', round(max, 2))

Análises

> Quantidade de transações por dia

* A quantidade de transações tem um pico próximo de 2500 transações diárias.
* A média e mediana estão bem próximas, com a média inferior a mediana, indicando uma distribuição assimetrica a esquerda (assimetria negativa).
* A distribuição não segue uma distribuição normal.
* Ha uma concentração muito grande de dias com quantidades entre 1996 e 2956.
* Dias com quantidade de transações menores que 2000 são consideradas outliers em relação ao comportamento da variável.

> Valor médio das transações

* A média das transações é de 43.42
* Existem transações com valores muito maiores acima da média (acima de 600).
* Transações com valores abaixo de 36.67 e acima de 49.34 são considerados outliers.

> Fraudes

* A quantidade média de fraudes por dia, quando ocorre, é de 8.79 e mediana é 7.
* Dias com quantidade de fraude acima de 12.5 são outliers
* 40% dos dias tiveram fraude.

### Análise Mensal

In [ ]:
df_time = pd.DataFrame()
df_time['year'] = df['date'].dt.to_period('Y')
df_time['year_month'] = df['date'].dt.to_period('M').dt.to_timestamp(how='start')
df_time['amount'] = df['amount']
df_time['target'] = df['target']
df_time.head()

In [ ]:
df_time_year = df_time.groupby(['year'])['amount'].sum().reset_index()

plt.subplots(figsize=(10, 5))
plt.title('Total das Transações por Ano')
sns.barplot(data=df_time_year, x='year', y='amount', alpha=0.5)
plt.ylabel('Valor (em milhões)')
plt.xlabel('Ano')

plt.show()

In [ ]:
df_time_year = df_time.groupby(['year','target'])['amount'].sum().reset_index()
df_time_year['amount_million'] = df_time_year['amount'] / 1000000
df_time_year.head()

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(121)
plt.title('Transações com Fraude')
ax1 = sns.barplot(data=df_time_year.query('target == "Yes"'), x='year', y='amount_million')
for patches in ax1.patches:
    ax1.annotate(f'{patches.get_height():.3f}', (patches.get_x() + patches.get_width() / 2., patches.get_height()), ha='center', va='center', xytext=(0, 5), textcoords='offset points')

plt.ylabel('Valor total das transações (milhão)')
plt.xlabel('Ano')

plt.subplot(122)
plt.title('Transações Válida')
ax2 = sns.barplot(data=df_time_year.query('target == "No"'), x='year', y='amount_million')
for patches in ax2.patches:
    ax2.annotate(f'{patches.get_height():.2f}', (patches.get_x() + patches.get_width() / 2., patches.get_height()), ha='center', va='center', xytext=(0, 5), textcoords='offset points')

plt.ylabel('Valor total das transações (milhão)')
plt.xlabel('Ano')

plt.suptitle('Total das transações por ano (Milhões)')
plt.tight_layout()
plt.show()

del df_time_year

In [ ]:
df_time_month = df_time.groupby(by=['year_month', 'target'])['amount'].agg(['sum', 'mean', 'std']).reset_index()
df_time_month['year'] = df_time_month['year_month'].dt.year
df_time_month['month'] = df_time_month['year_month'].dt.month
df_time_month['amount_million'] = df_time_month['sum'] / 1000000
df_time_month.head()

In [ ]:
for month in df_time_month['month'].unique():
  df_time_month_month = df_time_month.query('month == @month')

  plt.figure(figsize=(12,5))
  plt.suptitle(f'Valor das Transações no Mês: {month}')

  plt.subplot(121)
  plt.title('Transações com Fraude')

  ax = sns.barplot(data=df_time_month_month.query('target == "Yes"'), x='year', y='amount_million')

  for patches in ax.patches:
    ax.annotate(f'{patches.get_height():.3f}', (patches.get_x() + patches.get_width() / 2., patches.get_height()), ha='center', va='center', xytext=(0, 5), textcoords='offset points')

  mean = df_time_month_month.query('target == "Yes"')['amount_million'].mean()
  ax.axhline(mean, color='red', linestyle='--', label=f'Média: {mean:.3f}')
  ax.legend()

  plt.ylabel('Valor em Milhões')
  plt.xlabel('Ano')

  plt.subplot(122)
  plt.title('Transações Válida')
  sns.barplot(data=df_time_month_month.query('target == "No"'), x='year', y='amount_million')
  plt.ylabel('Valor em Milhões')
  plt.xlabel('Ano')

plt.tight_layout()
plt.show()

* Outubro e Dezembro são os meses com maior média de prejuizo nos anos avaliados (26 mil aproximadamente).
* 2016 foi o ano com maior quantidade e proporção de fraude.

## Análise Numérica

In [ ]:
number_columns = df.select_dtypes(include=['float64', 'int64']).columns
number_columns

In [ ]:
df[number_columns].describe()

In [ ]:
for column in filter(lambda x: x not in ['latitude', 'longitude'], number_columns):
  plot_hist_box(df, column)

In [ ]:
for column in filter(lambda x: x not in ['latitude', 'longitude'], number_columns):
    descritiva(df, column, 'target', map_target={ 'Yes': 1, 'No': 0 })

> amount

* Compras com valor abaixo de 75.71 possui menor indice de fraude

> current_age

* Faixa acima de 67 possui um maior indice de fraude

> retirement_age

* Indice de fraude muito volatil para o as idades de 53 a 79

> per_capita_income

* Indice de fraude maior para as faixas abaixo de 23553

> yearly_income

* Indice de fraude maior para as faixas abaixo de 58503

> total_debit

* Os dados dentro do range do boxplot possui maior indice de fraude

> credit_score

* O credit_score com faixa 703 a 731 possui uma maior taxa de indice de fraude.

> num_credit_cards

* Clientes com quantidade de cartão acima de 4 cartões tem uma taxa muito maior de indice de fraude

> credit_limit

* Clientes com limit de credito menor (abaixo de 6700) apresentam um maior indice de fraude



In [ ]:
ax = sns.pairplot(df[number_columns].drop(columns=['latitude', 'longitude']))
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Correlação entre as variáveis')

corr = df[number_columns].drop(columns=['latitude', 'longitude']).corr()
sns.heatmap(corr, annot=True, cmap='viridis', fmt='.2f')

plt.show()

In [ ]:
upper_triangle_indices = np.where(np.abs(np.triu(corr, k=1)) > 0.2)
colunas_indices = np.unique(np.concatenate(upper_triangle_indices))
colunas_filtradas = corr.columns[colunas_indices].tolist()
colunas_filtradas


In [ ]:
plt.figure(figsize=(12,8))
plt.title('Correlação entre as variáveis com maior correlação')

corr = df[colunas_filtradas].corr()
sns.heatmap(corr, annot=True, cmap='viridis', fmt='.2f')

plt.show()

* Correlação muito alta entre `per_capita_income` e `yearly_income` (0.95) indicando **multicolinearidade**.
* Correlação alta entre `credit_limit` e `per_capita_income` (0.61).
* Correlação alta entre `credit_limit` e `yearly_income` (0.58)

In [ ]:
# fig = px.scatter_mapbox(df,
#           lat="latitude",
#           lon="longitude",
#           hover_name="merchant_state",  # Informação ao passar o mouse
#           zoom=10,
#           color="merchant_state",
#           height=600)         # Altura do mapa

# # Define o estilo do mapa (opcional)
# fig.update_layout(mapbox_style="open-street-map")

# # Centraliza o mapa na média das coordenadas (opcional)
# fig.update_layout(
#     mapbox_center_lat=df['latitude'].mean(),
#     mapbox_center_lon=df['longitude'].mean()
# )

# # Exibe o mapa interativo
# fig.show()